In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils
from PIL import Image
from keras.models import load_model
import os,sys
import tensorflow as tf
from keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import random
import scipy
from scipy import ndimage
import math
import matlab
import matlab.engine

Using TensorFlow backend.
D:\Anaconda\envs\tensorflow-cpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tensorflow-cpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tensorflow-cpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tensorflow-cpu\lib\site-packages\tensorflo

In [2]:
model = ResNet50(weights=None,include_top=False)

D:\Anaconda\envs\tensorflow-cpu\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
model = tf.keras.models.load_model('small_degree_5.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
class Point(object):
    '''Creates a point on a coordinate plane with values x and y.'''

    COUNT = 0

    def __init__(self, x=0, y=0):
        '''Defines x and y variables'''
        self.X = x
        self.Y = y

    def move(self, dx, dy):
        '''Determines where x and y move'''
        self.X = self.X + dx
        self.Y = self.Y + dy

    def __str__(self):
        return "Point(%s,%s)"%(self.X, self.Y) 


    def getX(self):
        return self.X

    def getY(self):
        return self.Y

    def distance(self, other):
        dx = self.X - other.X
        dy = self.Y - other.Y
        return math.sqrt(dx**2 + dy**2)

    def testPoint(x=0,y=0):
        '''Returns a point and distance'''
        p1 = Point(3, 4)
        print(p1)
        p2 = Point(3,0)
        print(p2)
        # return math.hypot(dx, dy)
    
    # print ("distance = %s"%(testPoint()) )

In [6]:
class Line(object):
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
def GetLinePara(line):    
    line.a = line.p1.Y - line.p2.Y    
    line.b = line.p2.X - line.p1.X    
    line.c = line.p1.X * line.p2.Y - line.p2.X * line.p1.Y
def GetCrossPoint(l1,l2):    
    GetLinePara(l1)    
    GetLinePara(l2)    
    d = l1.a * l2.b - l2.a * l1.b    
    p = Point()    
    p.x = (l1.b * l2.c - l2.b * l1.c)*1.0 / d    
    p.y = (l1.c * l2.a - l2.c * l1.a)*1.0 / d
    return p

def GetCrossAngle(l1, l2):
    arr_0 = np.array([(l1.p2.X - l1.p1.X), (l1.p2.Y - l1.p1.Y)])
    arr_1 = np.array([(l2.p2.X - l2.p1.X), (l2.p2.Y - l2.p1.Y)])
    cos_value = (float(arr_0.dot(arr_1)) / (np.sqrt(arr_0.dot(arr_0)) * np.sqrt(arr_1.dot(arr_1))))   # 注意转成浮点数运算
    return np.arccos(cos_value) * (180/np.pi)


In [7]:
def length(a, b):
    d = math.sqrt(pow((a.X-b.X),2) + pow((a.Y-b.Y),2))
    return d

In [8]:
global l_pt
global r_pt

In [9]:
def _contain_red(image):
    count = 0
    for col in image[:,:,0]:
        for item in col:
            if item == 255:
                count += 1
            if count > 400:
                return True
    return False
            
def get_rotated(image):
    # shape图像的维度
    my_shape = image.shape
    # img[;,;,0] 0通道代表B 1代表G 2代表R
    # np.ones()*255, 即为255，255，255是白色
    white = np.ones(shape=my_shape, dtype=np.uint8)*255
    # 旋转角度
    global angle_rotated
    angle_rotated = np.random.randint(0,45)
    # 旋转
    rotated_image = imutils.rotate_bound(image,angle)
    rotated_white = imutils.rotate_bound(white,angle)
    (h_rotated,w_rotated,_) = rotated_image.shape
    '''
    plt.subplot(1,2,1)
    plt.imshow(rotated_image)
    plt.subplot(1,2,2)
    plt.imshow(rotated_white)
    plt.show()
    '''
    box_size = 224
    # 128*128 置零
    crop_image = np.zeros((box_size,box_size),dtype=np.uint8)
    # get a random upper left corner
    while True:
        pt_x = np.random.randint(h_rotated-box_size)
        pt_y = np.random.randint(w_rotated-box_size)
        # 判断四个坐标框起来的长方形在不在图像里 即是不是子图
        if all([rotated_white[pt_x,pt_y,0] == 255,
        rotated_white[pt_x+box_size,pt_y,0] == 255,
        rotated_white[pt_x,pt_y+box_size,0] == 255,
        rotated_white[pt_x+box_size,pt_y+box_size,0] == 255]):
            crop_image = rotated_image[pt_x:pt_x+box_size, pt_y:pt_y+box_size, :]
            # 一定会包含红线
            if _contain_red(crop_image):
                break
    #print("crop image size is ",crop_image.shape)        
    return crop_image
    #return cv2.equalizeHist(crop_image)

In [10]:
def draw_red_line(image,n):
    # 将线分成三个类别 斜率大于零 小于零 和 等于零
    # 存在随机数
    global l_pt
    global r_pt
    copy = np.copy(image)
    (h,w,c) = image.shape
    # left side random start pt generate
    mid_h = h//2
    range_h = np.random.randint(h//4)-h//6
    l_h = mid_h + range_h
    
    #l_pt = (np.random.randint(0,w//4),np.random.randint(0,h//4))
    l_pt = (np.random.randint(w//8,w//4),np.random.randint(h//6,h//3))
    # right side random start pt generate
    if n == 0:
        # upper
        r_h = np.random.randint(0,h//5-40)   
    elif n == 1:
        r_h = np.random.randint(h//5+40,h)
    elif n == 2:
        r_h = l_h
    r_pt = (w-1-np.random.randint(w//4,w//3),r_h) 

    return cv2.line(copy, l_pt, r_pt, (255,0,0), 3)

def draw_red_line_withcertain(image,n):
    # 将线分成三个类别 斜率大于零 小于零 和 等于零
    # 存在随机数
    global r_pt
    copy = np.copy(image)
    (h,w,c) = image.shape
    # left side random start pt generate
    mid_h = h//2
    range_h = np.random.randint(h//4)-h//6
    l_h = mid_h + range_h
    
    #l_pt = (np.random.randint(0,w//4),np.random.randint(0,h//4))
    
    # right side random start pt generate
    if n == 0:
        # upper
        r_h = np.random.randint(0,h//5-40)   
    elif n == 1:
        r_h = np.random.randint(h//5+40,h)
    elif n == 2:
        r_h = l_h
    r_pt = (w-1-np.random.randint(w//4,w//3),r_h) 

    return cv2.line(copy, l_pt, r_pt, (255,0,0), 3)

In [11]:
def draw_red_line_withangle(image,angle_last):
    # 将线分成三个类别 斜率大于零 小于零 和 等于零
    # 存在随机数
    global l_pt
    global r_pt
    copy = np.copy(image)
    (h,w,c) = image.shape
    # right side random start pt generate
    p1 = Point(l_pt[0],l_pt[1])
    angle_= np.random.uniform(angle_last-0.8,angle_last+0.8)
    angle_ = math.radians(angle_)
    # 上幅图的d
    #certain_d = np.random.randint(2*c//3,c)
    p2 = Point()
    p2.X = certain_d * (math.cos(angle_)) + p1.X
    p2.Y = certain_d * (math.sin(angle_)) + p1.Y
    if( p2.Y < -certain_d/3):
        p1.Y = p1.Y +certain_d/3
        p2.Y = p2.Y +certain_d/3
    l_pt = (int(p1.X + 0.5),int(p1.Y+0.5))
    r_pt = (int(p2.X+0.5),int(p2.Y+0.5))

    return cv2.line(copy, l_pt, r_pt, (255,0,0), 3)

In [12]:
def get_subim(image,pt_x,pt_y):
    box_size = 224
    crop_image = np.zeros((box_size,box_size),dtype=np.uint8)
    (h,w,_) = image.shape
    while True:
        
        if 224 == h:
            pt_y = 0
        else:
            pt_y = np.random.randint(h-box_size)
        while (pt_y + box_size > h):
            pt_y = np.random.randint(h-box_size)
        t = pt_x + box_size
        #if t > h:
        #    pt_x = pt_x -(t - h)
        if t > w:
            pt_x = pt_x - (t - w)
        crop_image = image[pt_y:pt_y+box_size, int(pt_x):int(pt_x)+box_size, :]
        
        if _contain_red(crop_image):
                break
        #pt_x = np.random.randint(h-box_size)
        
        #crop_image = image[pt_x:pt_x+box_size, pt_y:pt_y+box_size, :]
        #if _contain_red(crop_image):
         #       break
    return crop_image

In [13]:
def Geo_mag(image):
    # img = cv2.imread('real_dataset_raw/data (19).jpg',0)
    height,width = image.shape[:2]  #获取原图像的水平方向尺寸和垂直方向尺寸。
    #print(height)
    #print(image.shape)
    des = image
    while (height < 318) or (width < 318) :
        if height < 318 :
            ratio = 318 // height + 1
            des = cv2.resize(image,( width * ratio, height * ratio),interpolation=cv2.INTER_CUBIC)
            height,width = des.shape[:2]
            '''plt.subplot(1,2,1)
            plt.imshow(img)
            plt.subplot(1,2,2)
            plt.imshow(des)
            print(des.shape)
            plt.show()'''
        if width < 318 :
            ratio = 318 // width + 1
            des = cv2.resize(image,( width * ratio, height * ratio),interpolation=cv2.INTER_CUBIC)
            height,width = des.shape[:2]
        image = des
    height,width = image.shape[:2]  #获取原图像的水平方向尺寸和垂直方向尺寸。
    #print(height)
    #print(image.shape)
    return des
def Geo_mag_withoutrotate(image):
    # img = cv2.imread('real_dataset_raw/data (19).jpg',0)
    height,width = image.shape[:2]  #获取原图像的水平方向尺寸和垂直方向尺寸。
    #print(height)
    #print(image.shape)
    des = image
    while (height < 224) or (width < 224) :
        if height < 224 :
            ratio = 224 // height + 1
            des = cv2.resize(image,( width * ratio, height * ratio),interpolation=cv2.INTER_CUBIC)
            height,width = des.shape[:2]
            '''plt.subplot(1,2,1)
            plt.imshow(img)
            plt.subplot(1,2,2)
            plt.imshow(des)
            print(des.shape)
            plt.show()'''
        if width < 224 :
            ratio = 224 // width + 1
            des = cv2.resize(image,( width * ratio, height * ratio),interpolation=cv2.INTER_CUBIC)
            height,width = des.shape[:2]
        image = des
    height,width = image.shape[:2]  #获取原图像的水平方向尺寸和垂直方向尺寸。
    #print(height)
    #print(image.shape)
    return des


def get_subim_withoutpoint(image):
    box_size = 224
    crop_image = np.zeros((box_size,box_size),dtype=np.uint8)
    
    (h,w,_) = image.shape
    while True:
        pt_y = np.random.randint(h-box_size)
        pt_x = np.random.randint(w-box_size)
        
        t = pt_x + box_size
        #if t > h:
        #    pt_x = pt_x -(t - h)
        if t > w:
            pt_x = pt_x - (t - w)
        crop_image = image[pt_y:pt_y+box_size, int(pt_x):int(pt_x)+box_size, :]
        
        if _contain_red(crop_image):
                break
        #pt_x = np.random.randint(h-box_size)
        
        #crop_image = image[pt_x:pt_x+box_size, pt_y:pt_y+box_size, :]
        #if _contain_red(crop_image):
         #       break
    return crop_image

In [14]:
def getinitial(image_name):
    image2 = cv2.imread(image_name)
    des = Geo_mag_withoutrotate(image2)
    red_line = draw_red_line(des,1)

    return red_line

def getinitial_withcertain(image_name):
    image2 = cv2.imread(image_name)
    des = Geo_mag_withoutrotate(image2)
    red_line = draw_red_line_withangle(des,1)
    return red_line

In [17]:
import time

i_image = 0
path = 'sheng-control_processed'
write_path = path + '/' + 'resnet'
is_first = 1
global certain_d
while (i_image < 200):
    if(is_first == 1):
        red_line = getinitial(path+'/'+"%d.jpg"%(i_image+1))
        is_first = 0
    else:
        image2 = cv2.imread(path+'/'+"%d.jpg"%(i_image+1))
        des = Geo_mag_withoutrotate(image2)
        red_line = draw_red_line_withangle(des,theta)
        #red_line = getinitial_withcertain(path+'/'+"%d.jpg"%(i_image+1))
    image2 = cv2.imread(path+'/'+"%d.jpg"%(i_image+1))
    des = Geo_mag_withoutrotate(image2)
    i = 1
    p1 = Point(l_pt[0],l_pt[1])
    p2 = Point(r_pt[0],r_pt[1])
    theta_1 = 90
    theta_0 = -90
    start = time.time()
    i_1 = 0
    i_0 = 0
    data_0 = []
    data_1 = []
    pre_res = -1
    while i < 40:
        certain_d = length(p1,p2)
        d = certain_d
        angle = math.atan2((p2.Y-p1.Y), (p2.X-p1.X)) 
        theta = angle*(180/math.pi)
        #print("....",theta,".....")
        if pre_res == 1 :
            i_0 = 0
            i_1 = i_1 + 1
            if (theta < theta_1):
                if(theta == theta_0):
                    theta_0 = theta_0 - 2
                else:
                    theta_1 = theta
            if(theta_0 != -90):
                if(theta_1 - theta_0 < 0.1) &  (i_1 < 2) & (i_0 < 2):
                    #cv2.imwrite(write_path+'/'+"red_line_%d.jpg"%(i_image+1), red_line)
                    pre_angle = math.atan2((p2.Y-p1.Y), (p2.X-p1.X))
                    pre_theta = pre_angle*(180/math.pi)
                    f = open(write_path+'/'+"red_line_%d.txt"%(i_image+1),mode = 'w')
                    t = str(pre_theta)
                    f.write(t)
                    f.close()
                    break;
                elif (theta_1 - theta_0 < 0.5) & (i_1 > 1):
                    theta_0 = theta_0 - 1
                test = theta - (theta_1 - theta_0)/2
            else:
                test = theta - 3
        if pre_res == 0 :
            i_1 = 0
            i_0 = i_0 + 1
            if  (theta > theta_0):
                theta_0 = theta
            if(theta_1 != 90):
                if(theta_1 - theta_0 < 0.1)  &  (i_1 < 2) & (i_0 < 2):
                    #cv2.imwrite(write_path+'/'+"red_line_%d.jpg"%(i_image+1), red_line) 
                    pre_angle = math.atan2((p2.Y-p1.Y), (p2.X-p1.X))
                    pre_theta = pre_angle*(180/math.pi)
                    f = open(write_path+'/'+"red_line_%d.txt"%(i_image+1),mode = 'w')
                    t = str(pre_theta)
                    f.write(t)
                    f.close()
                    break;
                elif (theta_1 - theta_0 < 0.8) & (i_0 > 1):
                    theta_1 = theta_1 + 1
                else:
                    test = theta + (theta_1 - theta_0)/2
            else:
                test = theta + 3
        if pre_res == 2 :
            #cv2.imwrite(write_path+'/'+"red_line_%d.jpg"%(i_image+1), red_line) 
            pre_angle = math.atan2((p2.Y-p1.Y), (p2.X-p1.X))
            pre_theta = pre_angle*(180/math.pi)
            f = open(write_path+'/'+"red_line_%d.txt"%(i_image+1),mode = 'w')
            t = str(pre_theta)
            f.write(t)
            f.close()
            break
        # Update the coordinates of the line
        if pre_res != -1:
            angle_ = math.radians(test)
            p2.X = certain_d * (math.cos(angle_)) + p1.X
            p2.Y = certain_d * (math.sin(angle_)) + p1.Y
            if( p2.Y < -d/3):
                p1.Y = p1.Y +d/3
                p2.Y = p2.Y +d/3
            #print(test)
            l_pt = (int(p1.X + 0.5),int(p1.Y+0.5))
            r_pt = (int(p2.X+0.5),int(p2.Y+0.5))
            copy = np.copy(des)
            red_line = cv2.line(copy, l_pt, r_pt, (255,0,0), 3)
        k = 0
        first = 0
        class_0 = 0
        class_1 = 0
        class_2 = 0

        count_interval = int((p2.X-p1.X)/224 + 0.5)+1
        if count_interval%2 == 0:
            count_interval = count_interval +1
        satisfied_count = int(count_interval/2 + 0.5)
        interval = int((p2.X - p1.X)/count_interval+0.5)

        while k <count_interval:
            #The slope and intercept of the current reference line
            refer_k = (p2.Y-p1.Y)/(p2.X-p1.X)
            refer_b = p1.Y - refer_k * p1.X

            interval = int((p2.X - p1.X)/7+0.5)

            test_x = p1.X + interval*k
            test_y = refer_k*test_x + refer_b

            t = get_subim(red_line,int(test_x),int(test_y))
            cut = t / 255.0
            cut = np.expand_dims(cut, axis=0)
            t_k = np.argmax(model.predict(cut))
            if t_k == 0:
                class_0 = class_0+1 
            elif t_k == 1:
                class_1 = class_1+1
            else:
                class_2 = class_2+1
            if class_0 >= satisfied_count:
                t_k = 0
                data_t_0 = [i,theta]
                data_0.append(data_t_0)
                break;
            elif class_1 >= satisfied_count:
                t_k = 1
                data_t_1 = [i,theta]
                data_1.append(data_t_1)
                break;
            elif class_2 >= satisfied_count:
                t_k = 2
                break;
            else:
                k = 0
            k = k +1

        print(model.predict(cut))
        pre_res = t_k
        t_k = -1
        print(pre_res)
        #print(theta_0)
        #print(theta_1)
        i = i + 1
    i_image = i_image + 1
    #print("Reference line adjustment took {0} seconds".format(time.time()-start))


[[2.5677082e-12 1.0000000e+00 6.1822340e-12]]
1
[[1.2317867e-11 1.0000000e+00 1.1124663e-10]]
1
[[1.5754897e-09 1.0000000e+00 5.4832398e-08]]
1
[[9.3208096e-04 2.6702793e-02 9.7236520e-01]]
2
